In [1]:
import pennylane as qml
from pennylane import numpy as np
import QEMZMSEPC as qem

In [2]:
# we introduced noise into the network and trained it without any error mitigation techniques
# to observe the behavior of the Loss function degradation during training.

n_qubits = 2
dev = qml.device('default.mixed', wires=n_qubits)

nqubitspaulimatrices = qem.NqubitsPauliMatrices(n_qubits)
pauli_set_of_n_qubits = nqubitspaulimatrices.get_pauli_matrices_of_n_qubits()
nqubitschannel = qem.NqubitsChannel(n_qubits, pauli_set_of_n_qubits)

nqubitspaulichannel = nqubitschannel.nqubitsrandompaulichannel(p_identity=0.9)
nqubitsdepolarizingchannel = nqubitschannel.nqubitsdepolarizingchannel(0.9)

@qml.qnode(qml.device('default.mixed', wires=n_qubits))
def train_cir_without_qem(parameters):
    qml.StronglyEntanglingLayers(weights=parameters, wires=range(2))
    qml.QubitChannel(nqubitsdepolarizingchannel, wires=[0, 1])
    qml.QubitChannel(nqubitspaulichannel, wires=[0, 1])
    return qml.expval(qml.PauliZ(0)@qml.PauliZ(1))

shape = qml.StronglyEntanglingLayers.shape(n_layers=2, n_wires=2)
weights = np.random.random(size=shape)

def cost(x):
    return ((train_cir_without_qem(x))-(-1))**2

opt = qml.GradientDescentOptimizer(stepsize=0.1)

lst = []
steps = 100
params = weights

for i in range(steps):
    params = opt.step(cost, params)
    if (i + 1) % 5 == 0:
        print("Cost after step {:5d}: {: .7f}".format(i + 1, cost(params)))
        lst.append(cost(params))
print("Optimized rotation angles: {}".format(params))

Cost after step     5:  0.1735711
Cost after step    10:  0.0702897
Cost after step    15:  0.0457700
Cost after step    20:  0.0360291
Cost after step    25:  0.0311375
Cost after step    30:  0.0283360
Cost after step    35:  0.0265917
Cost after step    40:  0.0254413
Cost after step    45:  0.0246500
Cost after step    50:  0.0240884
Cost after step    55:  0.0236801
Cost after step    60:  0.0233777
Cost after step    65:  0.0231506
Cost after step    70:  0.0229780
Cost after step    75:  0.0228459
Cost after step    80:  0.0227439
Cost after step    85:  0.0226649
Cost after step    90:  0.0226035
Cost after step    95:  0.0225555
Cost after step   100:  0.0225179
Optimized rotation angles: [[[ 0.93524231  1.53927906  0.01304544]
  [ 0.6036347   1.28911111  0.0527892 ]]

 [[-0.05128551  1.85024235  0.17638228]
  [ 0.6259827   0.29750087  0.28238302]]]


In [3]:
#  It can be observed that after a certain number of iterations,
# the Loss function nearly stops decreasing,
# which is due to the limitation of the expressive capacity of the quantum network caused by noise.

In [4]:
# We estimate the total noise 'p_t' using the QEMZMSEPC method,
# allowing us to incorporate noise 'p_t' into the definition of the Loss function,
# thereby mitigating the impact of noise on quantum machine learning tasks.

operations = ['RX', 'RY', 'RZ', 'RX', 'RY', 'RZ', 'CNOT', 'CNOT',
              'RX', 'RY', 'RZ', 'RX', 'RY', 'RZ', 'CNOT', 'CNOT']
paras = [[0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 1], [1, 0],
         [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 1], [1, 0]]
# Due to our assumption that the noise channel is time-invariant,
# we randomly initialize a quantum circuit to estimate,
# with the estimation of  considered to be stable and unchanged in the short term.

qemzmsepc = qem.QEMZMSEPC(n_qubits)
_, p_t = qemzmsepc.qemzmsepc(operations=operations, paras=paras, dev=dev, p=0.9,
                             kraus_matrices_of_a_pauli_channel=nqubitspaulichannel)
print(p_t)

0.8503999658014473


In [5]:
@qml.qnode(qml.device('default.mixed', wires=n_qubits))
def train_cir_with_qem(parameters):
    qml.StronglyEntanglingLayers(weights=parameters, wires=range(2))
    qml.QubitChannel(nqubitsdepolarizingchannel, wires=[0, 1])
    qml.QubitChannel(nqubitspaulichannel, wires=[0, 1])
    return qml.expval(qml.PauliZ(0)@qml.PauliZ(1))

shape = qml.StronglyEntanglingLayers.shape(n_layers=2, n_wires=2)
weights = np.random.random(size=shape)

# We modify the definition of the Loss function
# by dividing the output of the quantum circuit by the noise parameter p_t
# to obtain the expected output of the quantum circuit after error mitigation,
# and then proceed with the Loss calculation.

def cost(x):
    return ((train_cir_with_qem(x)/p_t)-(-1))**2

opt = qml.GradientDescentOptimizer(stepsize=0.1)

lst = []
steps = 100
params = weights

for i in range(steps):
    params = opt.step(cost, params)
    if (i + 1) % 5 == 0:
        print("Cost after step {:5d}: {: .7f}".format(i + 1, cost(params)))
        lst.append(cost(params))
print("Optimized rotation angles: {}".format(params))

Cost after step     5:  0.6468637
Cost after step    10:  0.1300657
Cost after step    15:  0.0541426
Cost after step    20:  0.0306464
Cost after step    25:  0.0202435
Cost after step    30:  0.0146398
Cost after step    35:  0.0112289
Cost after step    40:  0.0089733
Cost after step    45:  0.0073901
Cost after step    50:  0.0062277
Cost after step    55:  0.0053440
Cost after step    60:  0.0046532
Cost after step    65:  0.0041007
Cost after step    70:  0.0036504
Cost after step    75:  0.0032775
Cost after step    80:  0.0029645
Cost after step    85:  0.0026985
Cost after step    90:  0.0024703
Cost after step    95:  0.0022727
Cost after step   100:  0.0021001
Optimized rotation angles: [[[0.88979539 1.05758663 0.03756295]
  [0.42941859 2.53512339 0.1052398 ]]

 [[0.08273137 0.58563469 0.35060925]
  [0.01258448 0.78138976 0.50541724]]]


In [6]:
# It can be seen that the Loss function after error mitigation performs better than without error mitigation.